### '아파트 청약'기사 크롤링 함수
- 내용에 '아파트 청약' 키워드가 포함된 기사 크롤링
    - 기간: 2020~2024
    - 합계: 91개

In [ ]:
import pandas as pd
import time
import requests
from bs4 import BeautifulSoup
import re
from tqdm import tqdm
import urllib.parse

In [ ]:
def generate_news_url(apartment_name: str) -> str:
    """아파트 청약 키워드 검색 URL 생성"""
    base_url = "https://search.naver.com/search.naver"
    query = 'query'
    
    params = {
        "where": "news",
        "query": query,
        "sm": "tab_opt",
        "sort": "1"  # 최신순 정렬
    }
    
    return base_url + "?" + urllib.parse.urlencode(params, quote_via=urllib.parse.quote)

In [11]:
import urllib.parse
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd

def generate_news_url(query: str, start: int, start_date: str, end_date: str) -> str:
    """네이버 뉴스 검색 URL 생성"""
    base_url = "https://search.naver.com/search.naver"
    params = {
        "where": "news",
        "query": query,
        "sm": "tab_opt",
        "sort": "1",  # 최신순 정렬
        "start": start,  # 페이지 시작 번호 (1, 11, 21...)
        "nso": f"so:dd,p:from{start_date}to{end_date},a:all"  # 날짜 범위 필터
    }
    return base_url + "?" + urllib.parse.urlencode(params, quote_via=urllib.parse.quote)

def crawl_naver_news(query: str, year: int, max_articles_per_year: int = 100) -> list:
    """네이버 뉴스에서 특정 키워드와 연도별 기사 크롤링"""
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }

    articles = []
    start = 1  # 네이버 뉴스 검색 결과의 첫 번째 페이지부터 시작
    start_date = f"{year}0101"  # 검색 시작 날짜 (YYYYMMDD)
    end_date = f"{year}1231"  # 검색 종료 날짜 (YYYYMMDD)

    while len(articles) < max_articles_per_year:
        try:
            # 검색 URL 생성
            search_url = generate_news_url(query, start, start_date, end_date)
            response = requests.get(search_url, headers=headers)
            response.raise_for_status()  # HTTP 오류 발생 시 예외 발생

            soup = BeautifulSoup(response.text, 'html.parser')

            # 네이버 뉴스 링크 필터링 (네이버 뉴스만)
            news_links = [
                a['href'] for a in soup.select('a.info')
                if '네이버뉴스' in a.text and 'news.naver.com' in a['href']
            ]

            if not news_links:  # 더 이상 기사가 없으면 종료
                print(f"{year}년에는 더 이상 기사가 없습니다.")
                break

            for article_url in news_links:
                if len(articles) >= max_articles_per_year:
                    break

                try:
                    # 개별 기사 페이지 요청
                    article_res = requests.get(article_url, headers=headers, timeout=5)
                    article_res.raise_for_status()
                    article_soup = BeautifulSoup(article_res.text, 'html.parser')

                    # 본문 추출
                    content_elem = article_soup.select_one('#newsct_article, #dic_area')
                    if content_elem:
                        content = content_elem.get_text(strip=True)
                    else:
                        continue

                    # 키워드 필터링 ("아파트 청약" 포함 여부 확인)
                    if "아파트 청약" not in content:
                        continue

                    # 제목 및 날짜 추출
                    title_elem = article_soup.select_one('#title_area')
                    date_elem = article_soup.select_one('.media_end_head_info_datestamp_time')

                    if title_elem and date_elem:
                        articles.append({
                            "title": title_elem.get_text(strip=True),
                            "date": date_elem['data-date-time'],
                            "content": content,
                            "url": article_url,
                            "year": year
                        })

                except Exception as e:
                    print(f"기사 처리 실패 ({article_url}): {str(e)}")

            start += 10  # 다음 페이지로 이동
            time.sleep(2)  # 요청 간 대기 시간 설정

        except Exception as e:
            print(f"페이지 처리 실패 ({search_url}): {str(e)}")
            break

    return articles


if __name__ == "__main__":
    query = "아파트 청약"

    print(f"{query} 관련 뉴스 크롤링 시작...")

    all_news_data = []

    for year in range(2020, 2025):  # 2020년부터 2024년까지 반복
        print(f"\n===== {year}년 크롤링 시작 =====")
        yearly_news_data = crawl_naver_news(query, year, max_articles_per_year=100)
        all_news_data.extend(yearly_news_data)
        print(f"{year}년 크롤링 완료: {len(yearly_news_data)}개의 기사 수집")

    print(f"\n총 {len(all_news_data)}개의 기사가 수집되었습니다.")

    # 결과 출력 및 저장
    df = pd.DataFrame(all_news_data)
    df.to_csv('2020_2024_아파트청약_뉴스_연도별_크롤링.csv', index=False, encoding='utf-8-sig')

    print("크롤링 결과가 '2020_2024_아파트청약_뉴스_연도별_크롤링.csv' 파일로 저장되었습니다.")

아파트 청약 관련 뉴스 크롤링 시작...

===== 2020년 크롤링 시작 =====
2020년에는 더 이상 기사가 없습니다.
2020년 크롤링 완료: 4개의 기사 수집

===== 2021년 크롤링 시작 =====
2021년에는 더 이상 기사가 없습니다.
2021년 크롤링 완료: 6개의 기사 수집

===== 2022년 크롤링 시작 =====
2022년에는 더 이상 기사가 없습니다.
2022년 크롤링 완료: 11개의 기사 수집

===== 2023년 크롤링 시작 =====
2023년에는 더 이상 기사가 없습니다.
2023년 크롤링 완료: 56개의 기사 수집

===== 2024년 크롤링 시작 =====
2024년에는 더 이상 기사가 없습니다.
2024년 크롤링 완료: 14개의 기사 수집

총 91개의 기사가 수집되었습니다.
크롤링 결과가 '2020_2024_아파트청약_뉴스_연도별_크롤링.csv' 파일로 저장되었습니다.


In [12]:
df = pd.read_csv('2020_2024_아파트청약_뉴스_연도별_크롤링.csv')
df.head()

,title,date,content,url,year
0,"부산, 올해 2만 5817가구 분양…강서·동래·부산진구에 집중",2020-12-31 19:08:43,부산 동래구 온천2구역 주택재개발정비사업 현장. 부산일보DB2021년 전국 민간 아...,https://n.news.naver.com/mnews/article/082/000...,2020
1,"[생생경제]청약, 기존 재고주택으로 분산 필요... 결국 집값 안정 전제되어야",2020-12-31 17:47:03,■ 방송 : YTN 라디오 FM 94.5 (15:10~16:00)■ 날짜 : 202...,https://n.news.naver.com/mnews/article/052/000...,2020
2,"""올해도 상한제 '로또 분양' 계속""",2020-12-31 16:29:06,"건설사 분양시장 전망3기신도시에 수요 일부 분산""재건축 규제 풀어 공급 늘려야""주요...",https://n.news.naver.com/mnews/article/015/000...,2020
3,주거용 오피스텔 찾는 실수요층...구리 갈매지구 ‘힐스테이트 갈매역 스칸센’ 관심,2020-12-31 14:24:21,"[아이뉴스24 이도영 기자] 최근 대출과 세금, 청약 규제 등으로 아파트 진입장벽이...",https://n.news.naver.com/mnews/article/031/000...,2020
4,"""메타버스부터 CCTV까지""…2022 랜선 해돋이 명소는?",2021-12-31 19:00:00,"유튜브 채팅창에 ""로또 1등"" ""다이어트 성공"" 기도지도앱 CCTV로 보는 새해 일...",https://n.news.naver.com/mnews/article/421/000...,2021
